In [2]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/02 17:48:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read the CSV files
df = spark.read.option("header", "true").csv("../shared/midterm/articles1.csv")

In [4]:
df.count()

50004

In [5]:
from datasketch import MinHash
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType

In [6]:
# Extract the reference article from the dataset
ref_article = df.filter(col("id") == "69716")

# Exclude the reference article from the dataset
df = df.filter(col("id") != "69716")

ref_article.count(), df.count()

(1, 50003)

In [7]:
ref_article.show()

df.show(5)

23/04/02 17:48:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, title, publication, author, date, year, month, url, content
 Schema: _c0, id, title, publication, author, date, year, month, url, content
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/midterm/articles1.csv


+-----+-----+--------------------+----------------+------+----------+------+-----+----+--------------------+
|  _c0|   id|               title|     publication|author|      date|  year|month| url|             content|
+-----+-----+--------------------+----------------+------+----------+------+-----+----+--------------------+
|50097|69716|California lifted...|Business Insider|  null|2016-05-21|2016.0|  5.0|null|’’ ’On Wednesday,...|
+-----+-----+--------------------+----------------+------+----------+------+-----+----+--------------------+

23/04/02 17:48:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, title, publication, author, date, year, month, url, content
 Schema: _c0, id, title, publication, author, date, year, month, url, content
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/midterm/articles1.csv
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
|_c0|   id|  

In [8]:
ref_article.head()["id"], ref_article.head()["content"]

23/04/02 17:48:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, title, publication, author, date, year, month, url, content
 Schema: _c0, id, title, publication, author, date, year, month, url, content
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/midterm/articles1.csv


23/04/02 17:48:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, title, publication, author, date, year, month, url, content
 Schema: _c0, id, title, publication, author, date, year, month, url, content
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/midterm/articles1.csv


('69716',
 '’’ ’On Wednesday, California  to its mandatory statewide water restrictions for urban use, a tip of the umbrella to the relatively wet winter northern parts of the state had, which helped  and brought a  to the Sierras.’ ’That decision was probably premature. For Southern California especially, the drought is still just as bad as ever. In much of the Sierra Nevada and Southern California, there’s still two or three entire years of rain missing since this drought began five years ago.’ ’If that’s not a reason to continue with   water conservation, I don’t know what is. In making the announcement, the state declared that local communities could set their own limits, so the opposite will likely happen as they back off on restrictions for things like watering lawns.’ ’Add climate change on top, and California is still heading toward drought disaster.  means the state’s crucial snowpack now melts much faster than in the past, creating a false sense of security before running out

In [9]:
# Create a MinHash for the reference article
reference_minhash = MinHash(num_perm=128)
for word in ref_article.head()["content"].split():
    reference_minhash.update(word.encode("utf-8"))

23/04/02 17:48:38 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, title, publication, author, date, year, month, url, content
 Schema: _c0, id, title, publication, author, date, year, month, url, content
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/midterm/articles1.csv


In [10]:
# Function to calculate the estimated Jaccard similarity
def calculate_jaccard_similarity(article_content):
    if article_content is not None:
        article_minhash = MinHash(num_perm=128)
        for word in article_content.split():
            article_minhash.update(word.encode("utf-8"))
        estimated_jaccard = reference_minhash.jaccard(article_minhash)
        return estimated_jaccard

# Register the UDF
jaccard_similarity_udf = udf(calculate_jaccard_similarity, FloatType())

In [11]:
# Calculate the Jaccard similarity for each article in the DataFrame
jaccard_estimations_df = df.withColumn("jaccard_estimation", jaccard_similarity_udf(col("content")))

In [12]:
# Get the top 10 articles with the highest Jaccard similarity
jaccard_estimations_df.orderBy(col("jaccard_estimation").desc()).show(10)

23/04/02 17:52:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, title, publication, author, date, year, month, url, content
 Schema: _c0, id, title, publication, author, date, year, month, url, content
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/midterm/articles1.csv


+-----+-----+--------------------+----------------+--------------+----------+------+-----+----+--------------------+------------------+
|  _c0|   id|               title|     publication|        author|      date|  year|month| url|             content|jaccard_estimation|
+-----+-----+--------------------+----------------+--------------+----------+------+-----+----+--------------------+------------------+
| 7186|25722|The Most Importan...|  New York Times|     Nate Cohn|2016-04-21|2016.0|  4.0|null|Join us for New Y...|          0.171875|
|34293|53109|Police shootout s...|             CNN|   Ana Cabrera|2017-01-27|2017.0|  1.0|null|Denver, Colorado ...|         0.1640625|
|47365|66574|Tesla fans are ab...|Business Insider|Matthew DeBord|2017-04-23|2017.0|  4.0|null|’’ ’Tesla is pois...|         0.1640625|
|16833|35570|Pastor Mark Burns...|       Breitbart|  John Hayward|2016-08-29|2016.0|  8.0|null|“This is a really...|         0.1640625|
|49301|68777|Trump is making a...|Business Insid